In [1]:
import os
# directory_path/label_folder/video.mp4 (or .mov, .MOV)
directory_path = "Data - Vòng loại/" # Thư mục gốc gồm các folder cho mỗi nhãn, chứa các video
save_directory = "Processed data txt/" # Thư mục chứa output là danh sách skeleton keypoint cho từng frame

In [2]:
import mediapipe as mp
import cv2
pose = mp.solutions.pose.Pose()

def calculate_bounding_box(landmarks):
    min_x = min(landmark.x for landmark in landmarks)
    min_y = min(landmark.y for landmark in landmarks)
    max_x = max(landmark.x for landmark in landmarks)
    max_y = max(landmark.y for landmark in landmarks)

    return BoundingBox(min_x, min_y, max_x, max_y)

class BoundingBox:
    def __init__(self, top, left, bottom, right):
        self.top = top
        self.left = left
        self.bottom = bottom
        self.right = right

def recalculate_landmarks(landmarks, bounding_box):
  for landmark in landmarks:
    landmark.x = (landmark.x - bounding_box.top)/(bounding_box.bottom - bounding_box.top)
    landmark.y = (landmark.y - bounding_box.left)/(bounding_box.right - bounding_box.left)
  return landmarks

def make_landmark_timestep(landmarks):
  c_lm = []
  for landmark in landmarks:
    c_lm.append(landmark.x)
    c_lm.append(landmark.y)
    c_lm.append(landmark.z)
    c_lm.append(landmark.visibility)
  return c_lm

def load_video(video_path): # mỗi video trả về một list result_landmarks cho mỗi video
    cap = cv2.VideoCapture(video_path)
    result_landmarks = []
    while cap.isOpened():
        # Đọc frame
        ret, frame = cap.read()
        if not ret:
          break
        # Chuyển đổi frame sang RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # Xác định pose_skeleton
        results = pose.process(frame_rgb)
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            # Xác định bao tương đối của skeleton
            bounding_box = calculate_bounding_box(landmarks)
            # Resize skeleton để trải rộng trong phạm vi [0,1]
            resized_landmarks = recalculate_landmarks(landmarks, bounding_box)
            # Lưu vào tập result_landmarks:
            result_landmarks.append(make_landmark_timestep(resized_landmarks))
    cap.release()
    return result_landmarks # sau đó ghi vào csv


In [5]:
import pandas as pd

def loop_data_for_label(directory_path,save_directory):
  list_label = os.listdir(directory_path)
  for label_folder in list_label:
    subpath = os.path.join(directory_path,label_folder)
    save_subpath = os.path.join(save_directory,label_folder)
    count = 0
    video_list = os.listdir(subpath)
    for video in video_list:
      count+=1
      extracted_data = load_video(os.path.join(subpath,video))
      df = pd.DataFrame(extracted_data)
      df.to_csv(save_subpath+"/"+str(count)+".txt")
    print("processed "+label_folder)

In [6]:
list_label = os.listdir(directory_path)
loop_data_for_label(directory_path,save_directory)
print("Data processed")

processed decline bench press
processed hammer curl
processed hip thrust
processed incline bench press
Data processed
